# Packaging Model

## Agenda
- Package the given model using Torch Model Archive
- Write a custom handler to support pre processing and post processing

In [1]:
!ls ../artifacts/model/

amazon	amazon_trace


In [2]:
!ls ../artifacts/model/amazon

config.json	    setup_config.json	     tokenizer_config.json
index_to_name.json  special_tokens_map.json  training_args.bin
pytorch_model.bin   tokenizer.json	     vocab.txt


In [3]:
!ls ../artifacts/model/amazon_trace

index_to_name.json  special_tokens_map.json  traced_model.pt
model_store	    tokenizer.json	     vocab.txt
setup_config.json   tokenizer_config.json


In [4]:
%%bash 

torch-model-archiver --help

usage: torch-model-archiver [-h] --model-name MODEL_NAME
                            [--serialized-file SERIALIZED_FILE]
                            [--model-file MODEL_FILE] --handler HANDLER
                            [--extra-files EXTRA_FILES]
                            [--runtime {python,python2,python3}]
                            [--export-path EXPORT_PATH]
                            [--archive-format {tgz,no-archive,default}] [-f]
                            -v VERSION [-r REQUIREMENTS_FILE]

Torch Model Archiver Tool

optional arguments:
  -h, --help            show this help message and exit
  --model-name MODEL_NAME
                        Exported model name. Exported file will be named as
                        model-name.mar and saved in current working directory if no --export-path is
                        specified, else it will be saved under the export path
  --serialized-file SERIALIZED_FILE
                        Path to .pt or .pth file containing state_dic

In [5]:
%%bash
MODEL_NAME="pt_classifier"
cd .. 

echo torch-model-archiver --model-name ${MODEL_NAME} 

torch-model-archiver --model-name pt_classifier


In [6]:
!ls ../artifacts/model/amazon_trace/

index_to_name.json  special_tokens_map.json  traced_model.pt
model_store	    tokenizer.json	     vocab.txt
setup_config.json   tokenizer_config.json


In [7]:
!ls ../artifacts/model/amazon_trace

index_to_name.json  special_tokens_map.json  traced_model.pt
model_store	    tokenizer.json	     vocab.txt
setup_config.json   tokenizer_config.json


In [8]:
%%bash

cd ..
pwd

ARTIFACT_BASE_DIR="artifacts/model/amazon_trace"

MODEL_NAME="pt_classifier"
MODEL_VERSION="1.0"
MODEL_STORE="${ARTIFACT_BASE_DIR}/model_store"
MODEL_SERIALIZED_FILE="${ARTIFACT_BASE_DIR}/traced_model.pt"

TOKENIZER_FILES="${ARTIFACT_BASE_DIR}/tokenizer_config.json,${ARTIFACT_BASE_DIR}/special_tokens_map.json,${ARTIFACT_BASE_DIR}/vocab.txt,${ARTIFACT_BASE_DIR}/tokenizer.json"
MODEL_EXTRA_FILES="${ARTIFACT_BASE_DIR}/index_to_name.json,${ARTIFACT_BASE_DIR}/setup_config.json,${TOKENIZER_FILES}"




mkdir -p $MODEL_STORE

torch-model-archiver --model-name ${MODEL_NAME} \
--version ${MODEL_VERSION} \
--serialized-file ${MODEL_SERIALIZED_FILE} \
--export-path ${MODEL_STORE} \
--extra-files ${MODEL_EXTRA_FILES} \
--handler ./serving/handler.py \
--force



/home/jupyter/tutorials/personal/pydata_bert


WARNING - Overwriting artifacts/model/amazon_trace/model_store/pt_classifier.mar ...


In [9]:
%%bash
cd .. 
cp artifacts/model/amazon_trace/model_store/* serving/model_store
cp artifacts/model/amazon_trace/setup_config.json serving/model_store/

In [10]:
%%bash
cd ..
echo torchserve --ts-config ./serving/config.properties \
--start --model-store ./serving/model_store --ncs



torchserve --ts-config ./serving/config.properties --start --model-store ./serving/model_store --ncs


In [ ]:
torchserve --stop

In [ ]:
!curl "http://localhost:9081/models"



In [ ]:
curl -X POST http://127.0.0.1:8080/predictions/my_tc -T Seq_classification_artifacts/sample_text_captum_input.txt

In [ ]:
!curl http://localhost:9081/models/pt_classifier

In [ ]:
! curl -X POST http://localhost:9080/predictions/pt_classifier \
        -H 'Content-Type: application/json' \
        -d '{"text":"herbal tea","request_id":"test_id"}'

In [ ]:
%%bash
cd ..
curl -X POST http://localhost:9080/predictions/pt_classifier \
        -H 'Content-Type: application/json' \
        -d @serving/sample_input.json

In [ ]:
! curl -X POST http://localhost:9080/explanations/pt_classifier \
        -H 'Content-Type: application/json' \
        -d '{"text":"herbal tea","request_id":"test_id"}'